In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
np.random.seed(1234)

df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                   'key2': ['one', 'two', 'one', 'two', 'one'],
                   'data1': np.random.randn(5),
                   'data2' : np.random.randn(5)
                  })

In [3]:
df

,data1,data2,key1,key2
0,0.471435,0.887163,a,one
1,-1.190976,0.859588,a,two
2,1.432707,-0.636524,b,one
3,-0.312652,0.015696,b,two
4,-0.720589,-2.242685,a,one


In [13]:
group1 = df.groupby(['key1','key2'])['data1'].mean()
group1

key1  key2
a     one    -0.124577
      two    -1.190976
b     one     1.432707
      two    -0.312652
Name: data1, dtype: float64

In [14]:
# convert it to a pivot table

In [15]:
group1.unstack()

key2,one,two
key1,,
a,-0.124577,-1.190976
b,1.432707,-0.312652


In [39]:
group1.unstack('key1')

key1,a,b
key2,,
one,-0.124577,1.432707
two,-1.190976,-0.312652


In [16]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

# Iterating Over Groups
    The GroupBy object supports iteration, generating a sequence of 2-tuples containing the group name along with the chunk of data. 
    It is useful to compute a dict of the data pieces as a one-liner:

In [17]:
for name, group in df.groupby('key1'):
    print name
    print group

a
      data1     data2 key1 key2
0  0.471435  0.887163    a  one
1 -1.190976  0.859588    a  two
4 -0.720589 -2.242685    a  one
b
      data1     data2 key1 key2
2  1.432707 -0.636524    b  one
3 -0.312652  0.015696    b  two


In [21]:
# example
tuple_example = (1, 3),(2 ,4)
list(tuple_example)

[(1, 3), (2, 4)]

In [22]:
dict(list(tuple_example))

{1: 3, 2: 4}

In [18]:
# for example
pieces = dict(list(df.groupby('key1')))

In [19]:
pieces['b']

,data1,data2,key1,key2
2,1.432707,-0.636524,b,one
3,-0.312652,0.015696,b,two


# Grouping with Dicts and Series

In [24]:
people = pd.DataFrame(np.random.randn(5, 5),
                    columns=['a', 'b', 'c', 'd', 'e'],
                    index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people

,a,b,c,d,e
Joe,1.150036,0.991946,0.953324,-2.021255,-0.334077
Steve,0.002118,0.405453,0.289092,1.321158,-1.546906
Wes,-0.202646,-0.655969,0.193421,0.553439,1.318152
Jim,-0.469305,0.675554,-1.817027,-0.183109,1.058969
Travis,-0.397840,0.337438,1.047579,1.045938,0.863717


In [26]:
people.loc[['Steve','Wes'],['b','c']] = np.nan
people

,a,b,c,d,e
Joe,1.150036,0.991946,0.953324,-2.021255,-0.334077
Steve,0.002118,NaN,NaN,1.321158,-1.546906
Wes,-0.202646,NaN,NaN,0.553439,1.318152
Jim,-0.469305,0.675554,-1.817027,-0.183109,1.058969
Travis,-0.397840,0.337438,1.047579,1.045938,0.863717


In [30]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}
by_column = people.groupby(mapping, axis = 1) # has to put axis = 1
by_column.count()

,blue,red
Joe,2,3
Steve,1,2
Wes,1,2
Jim,2,3
Travis,2,3


# Grouping with Functions

    More concretely, consider the example DataFrame from the previous section, which has people’s first names as index values. Suppose you wanted to group by the length of the names; you could compute an array of string lengths, but instead you can just pass the len function:
    Mixing functions with arrays, dicts, or Series is not a problem as everything gets con- verted to arrays internally:

In [ ]:
# example1

In [31]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.202646  0.991946  0.953324 -2.021255 -0.334077
  two -0.469305  0.675554 -1.817027 -0.183109  1.058969
5 one  0.002118       NaN       NaN  1.321158 -1.546906
6 two -0.397840  0.337438  1.047579  1.045938  0.863717

In [ ]:
#example2

In [33]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [34]:
grouped = df.groupby('key1')
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.662411,3.129848
b,1.745359,0.652220


In [37]:
df.groupby('key1',as_index = False).agg(peak_to_peak)

,key1,data1,data2
0,a,1.662411,3.129848
1,b,1.745359,0.652220


In [35]:
grouped.agg(['mean','std',peak_to_peak])

data1                            data2                       
          mean       std peak_to_peak      mean       std peak_to_peak
key1                                                                  
a    -0.480043  0.856913     1.662411 -0.165311  1.799111     3.129848
b     0.560028  1.234155     1.745359 -0.310414  0.461189     0.652220

In [36]:
#rename your aggregation function
grouped.agg([('foo', 'mean'), ('bar', np.std)])

data1               data2          
           foo       bar       foo       bar
key1                                        
a    -0.480043  0.856913 -0.165311  1.799111
b     0.560028  1.234155 -0.310414  0.461189

# example 3 --merge groupby with df

In [40]:
df

,data1,data2,key1,key2
0,0.471435,0.887163,a,one
1,-1.190976,0.859588,a,two
2,1.432707,-0.636524,b,one
3,-0.312652,0.015696,b,two
4,-0.720589,-2.242685,a,one


In [42]:
group_mean = df.groupby('key1').mean().add_prefix('mean_')
group_mean

,mean_data1,mean_data2
key1,,
a,-0.480043,-0.165311
b,0.560028,-0.310414


In [43]:
pd.merge(df, group_mean, left_on='key1',right_index = True)

,data1,data2,key1,key2,mean_data1,mean_data2
0,0.471435,0.887163,a,one,-0.480043,-0.165311
1,-1.190976,0.859588,a,two,-0.480043,-0.165311
4,-0.720589,-2.242685,a,one,-0.480043,-0.165311
2,1.432707,-0.636524,b,one,0.560028,-0.310414
3,-0.312652,0.015696,b,two,0.560028,-0.310414


In [44]:
# another good way to use groupby
df.groupby('key1')['data1'].describe()

key1       
a     count    3.000000
      mean    -0.480043
      std      0.856913
      min     -1.190976
      25%     -0.955782
      50%     -0.720589
      75%     -0.124577
      max      0.471435
b     count    2.000000
      mean     0.560028
      std      1.234155
      min     -0.312652
      25%      0.123688
      50%      0.560028
      75%      0.996367
      max      1.432707
Name: data1, dtype: float64

In [45]:
# unstack to this way is better to have a clearer comparison
df.groupby('key1')['data1'].describe().unstack('key1')

key1,a,b
count,3.000000,2.000000
mean,-0.480043,0.560028
std,0.856913,1.234155
min,-1.190976,-0.312652
25%,-0.955782,0.123688
50%,-0.720589,0.560028
75%,-0.124577,0.996367
max,0.471435,1.432707


# pd.qcut

In [ ]:
# assign them with 3 labels: not_pop, so_so, popular
purchase_categories = ["not_popular","so_so","very_popular"]
# sort_item['popularity'] = pd.qcut(sort_item['cnt_purchase'],len(purchase_categories),purchase_categories)
shoes_df['popularity'] = pd.qcut(shoes_df['cnt_purchase'], len(purchase_categories), labels= purchase_categories)

In [ ]:
# Pandas has a bult-in function that will perform this calculation
        # This will give the bottom 0% to 10% of students the grade 'F',
        # 10% to 20% the grade 'D', and so on. You can read more about
        # the qcut() function here:

pd.qcut(exam_grades,  [0, 0.1, 0.2, 0.5, 0.8, 1],  labels=['F', 'D', 'C', 'B', 'A'])


# Pivot_table

In [46]:
df

,data1,data2,key1,key2
0,0.471435,0.887163,a,one
1,-1.190976,0.859588,a,two
2,1.432707,-0.636524,b,one
3,-0.312652,0.015696,b,two
4,-0.720589,-2.242685,a,one


In [49]:
pd.pivot_table(data = df, index ='key1', columns= 'key2', values= 'data1',aggfunc='count') 
# aggfunc = 'mean' default

key2,one,two
key1,,
a,2,1
b,1,1
